# Confounder Template: *Multinomial Naive Bayes Edition*
---

### Introduction

This a "blank slate" version of [Confounder](https://github.com/analyticascent/confounder) that can be repurposed for whatever subject matter you choose. To use it, you need to produce a starting framework consisting of a **list criteria** of possible *omissions* to look for that are related to the topic of your choosing. Then you need to gather **sample articles** of text that *do* and *do not* meet the various criteria you came up with. Note that each of the criteria you have listed has to be checked independently from the others in order to ensure greater accuracy.

Here the term *articles* and/or *text samples* will be used throughout this notebook file to refer to the passages of text put under scrutiny.

Your collection of text samples will be the **training set** of raw text that [Confounder](https://github.com/analyticascent/confounder) uses to learn distinctions between "accurate" versus "inaccurate" coverage of a topic. To be effective, the collection must be *representative* of things that meet or fail the criteria you choose. *Representative* in this context means articles on a topic that do make one of the errors you have listed as well as samples that do not in as varied of ways as commonly possible. That set could be academic studies, policy papers, news articles, blog posts, even social media posts. 

**Bear in mind that there is no such thing as true "fact-checking" software - anyone claiming otherwise is peddling BS.** [Confounder](https://github.com/analyticascent/confounder) is simply a means of *consolidating* the validation of whether text passages meet pre-defined criteria. That *criteria* can be open to interpretation, as well as whether a training sample *meets or fails* parts of that criteria. And yes, these problems exist in spam filtering as well (of which [Confounder](https://github.com/analyticascent/confounder) is a functional derivative).

To reduce bias, [Confounder](https://github.com/analyticascent/confounder) is a project geared at detecting *omitted variables* - which is far different from trying to deduce the truth value of statements.

To reduce these problems, you need to deduce your framework from universally objective rules for accurate research methodology. This image below sums up some of the most fundamental errors any "X causes Y" claims need to avoid, and fitting this to a particular political, economic, or scientific issue with minimal value assumptions should be high priority for [Confounder](https://github.com/analyticascent/confounder) users:

![Methodological Criteria for Accurate Research](https://raw.githubusercontent.com/analyticascent/confounder/master/Research%20Methodology.png)

Rather than criticize [Confounder](https://github.com/analyticascent/confounder), anyone objecting to conclusions promoted with this tool should critique the *actual framework* used to *train* the classifier.

The effectiveness of the code used for this Confounder template is only as good as the *framework* you use to find those kinds of errors. The list of errors and missing variables must be comprehensive but avoid overlap, and the sample articles must be large in number and representative of those unique errors. This will better allow the Confounder notebook scripts to "learn" how accurate and inaccurate articles differ from each other.

You can think of the operational nature of this notebook as being a spam filter. You feed in examples of article text that does or does not meet the various criteria in your list. The raw text is then converted to *document-term matrices* which will use the frequency of unique words to distinguish things that meet or fail various criteria. A variant of **Naive Bayes** that treats each word as a variable will then deduce whether *new text samples* are more similar to things meeting or failing a given criteria. These steps are ellaborated on in the notebook's documentation below, so read on to learn about usage.

---
## Step 1: Importing Needed Libraries

You will need *pandas* to read in rows and colums (containing the raw article text, and columns for all of the criteria of interest. A **0** in a column indicates the criteria was not met, a **1** indicates it was sufficient.

*Numpy* and *scipy* add functionality that you will depend on throughout notebook use. Very specific tools are also imported from *scikit-learn.* Additionally, a few natural language processing tools are imported which may be used to boost model accuracy (with iterative trial and error).

In [ ]:
# These are the "libraries" you need to import to run the scripts that follow. It may take several seconds for some to load.

import pandas as pd
import numpy as np  # pandas, numpy, and scipy are core libraries cells that follow will depend on for a wide variety of things
import scipy as sp

from sklearn.cross_validation import train_test_split  # train_test_split will all you to check accuracy on existing data
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer  # these structure text into word vectors

# CountVectorizer turns raw text into word frequency counts, aka "bag of words." Very simple way of structuring text
# TfidVectorizer is a little more complex. "Term Frequency-Inverse Document Frequency" will find relative importance of terms

from sklearn.naive_bayes import MultinomialNB  # multinomial naive bayes will classify text samples from the vectorizer results
from sklearn import metrics  # metrics will be used to evaluate the accuracy of the model when you run train_test_split

---
## Step 2: Reading in the CSV File Containing the Training Corpus

As previously mentioned, you will need some labeled articles in CSV form to train the classifier. **This is how *supervised machine learning* is supposed to work - the "label" columns of what criteria the sample text failed or met are the categories we are aiming to classify articles under.** The classifier (Naive Bayes in this case) needs to have a sample of various articles to compare future articles against. It is strongly advised that you spend time gathering a varied and large sample of articles that meet or fail as wide a variety of criteria as possible, otherwise Confounder may not be able to predict new samples accurately.

In [ ]:
# Read post_feed.csv into a DataFrame. Any CSV with columns containing raw tweet contents and usernames can often work.
# If you're offline, replace the link with the file location for post_feed.csv if you have it stored locally.

url = ''
data = pd.read_csv(url)

The cells below will be used to verify if the CSV file has loaded properly. You need to have a CSV file with properly labeled columns and rows containing the raw article text (first column), and then a **0** or a **1** within each criteria column. The CSV file is basically a working version of the framework you put together.

In [ ]:
# check the first five rows/tweets

data.head()

In [ ]:
# check the first five rows in a shorter format

X.head()

In [ ]:
# check the number of rows and columns

X.shape

If the rows and columns that appeared above appear to have loaded properly (column titles and data in proper places), proceed with the next steps.

---
## Step 3: Define Variables for *train_test_split* Accuracy Experiments

Given the contents of the raw article text, did it likely meet or fail a criteria feature? **We will do a train/test split with the training data to measure predictive accuracy for each criteria column.** Define **X** as the raw article text (the manipulated variable), and some **y** variables as the individual criteria columns you are trying to classify future text into.

In [ ]:
# One change you might make below is defining more possible y variables to test accuracy for.
# column_1 and so on will likely need to be changed to match your CSV column titles.


X = data.raw_text  # Depending on the raw article text column contents...
y1 = data.column_1  # ...how likely did it meet this criteria?
y2 = data.column_2  # Or this next criteria column?
y3 = data.column_3  # And so on depending on your CSV file.

# Below splits the new DataFrame into training and testing sets

X_train, X_test, y1_train, y1_test = train_test_split(X, y1, random_state=1)

# it may not be a bad idea to try out different random_state values in your testing

You now have the columns of the CSV file defined as variables. No you just need to *vectorize* the raw_text you have stored in the **X** variable. 

---
## Step 4: Use *CountVectorizer* to Turn *X_train* and *X_test* into Document-Term Matrices

- **What:** Turn the training and testing portions of your framework samples into *document-term matrices*
- **Why:** Gives structure to previously unstructured text; you now have word frequency counts
- **Notes:** Easier with English text, not easy with langauges where beginning/end of words or sentences is ambigous

We are now going to create what are called *document-term matrices* of the sample articles. **Think of these as *rows and columns* which store numbers representing *how often* certain terms appear in different text samples.** See the image below to better understand what that looks like:

&nbsp;

![Document-Term Matrix](http://mlg.postech.ac.kr/static/research/nmf_cluster1.PNG)

&nbsp;

In [ ]:
# use CountVectorizer to create document-term matrices from X_train and X_test

vect = CountVectorizer()  # because vect is way easier to type than CountVectorizer
X_train_dtm = vect.fit_transform(X_train)  # stores a vectorized X_train sample into X_train_dtm
X_test_dtm = vect.transform(X_test)  # stores a vectorized X_train sample into X_train_dtm

# now we have quantitative info about articles that classifier can work with

**Just to clarify what's going on in the adjacent cells:** All the **rows** are of course the *individual tweets* that are stored in the CSV file. But the astronomical crapload of **columns** is literally *each unique term* that appears. Those are going to be the "features" used to "fingerprint" one user from another. 

In [ ]:
vect

In [ ]:
# rows are documents, columns are terms (aka "tokens" or "features")

X_train_dtm.shape

In [ ]:
# last 50 features

print vect.get_feature_names()[-50:]

In [ ]:
# show vectorizer options
vect

[CountVectorizer documentation](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

- Parameter **lowercase:** boolean, True by default
    - If True, Convert all characters to lowercase before tokenizing.

In [ ]:
# We will not convert to lowercase this time, but if we did it would reduce the number of quantified features

vect = CountVectorizer(lowercase=False)
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm.shape

In [ ]:
# last 50 features

print vect.get_feature_names()[-50:]

Below, the cell will allow you to augment how *CountVectorizer* works by including a range of **n-grams.** These are *word sequences,* so a 2-gram for instance will be a *pair* of words. The result from including *that* range is that the resulting *document-term matrices* will contain frequency counts of how often pairs of words appear, as well as single terms if you specify so as a parameter. 

- Parameter **ngram_range:** tuple (min_n, max_n)
    - The lower and upper boundary of the range of n-values for different n-grams to be extracted. All values of n such that min_n <= n <= max_n will be used.

In [ ]:
# include 1-grams and 2-grams

vect = CountVectorizer(ngram_range=(1, 2))  # sets the vectorizer to look at single as well as pairs of words
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm.shape

In [ ]:
# last 50 features

print vect.get_feature_names()[-50:]

---
## Step 5: Test Predictive Accuracy for First Criteria Feature

How accurate can we predict whether a text sample met or failed a criteria compared to the way you originally labeled it?

In [ ]:
# use default options for CountVectorizer

vect = CountVectorizer()


# create document-term matrices using CountVectorizer, and store them as variables

X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)


# use Naive Bayes to predict the first feature of the list criteria

nb = MultinomialNB()
nb.fit(X_train_dtm, y1_train)
y_pred_class = nb.predict(X_test_dtm)


# calculate accuracy

print metrics.accuracy_score(y1_test, y_pred_class)

The cell below will eliminate the need for typing in the same code over and over again, as well as produce an output that includes all the information we need to know about how the number of unique features is affecting the classifier accuracy.

In [ ]:
# define a function that accepts a vectorizer and calculates the accuracy

def tokenize_test(vect):
    X_train_dtm = vect.fit_transform(X_train)
    print 'Features: ', X_train_dtm.shape[1]  # this output will be unique words or n-grams
    X_test_dtm = vect.transform(X_test)
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y1_train)
    y_pred_class = nb.predict(X_test_dtm)
    print 'Accuracy: ', metrics.accuracy_score(y1_test, y_pred_class)

In [ ]:
vect = CountVectorizer()
tokenize_test(vect)

---

## Step 6: Test Stopword Removal and N-Grams to Potentially Boost Accuracy

In [ ]:
# include 1-grams and 2-grams

vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test(vect)

- **What:** Remove common words that will likely appear in any text
- **Why:** They don't tell you much about your text

In [ ]:
# show vectorizer options

vect

- **stop_words:** string {'english'}, list, or None (default)
    - If 'english', a built-in stop word list for English is used.
    - If a list, that list is assumed to contain stop words, all of which will be removed from the resulting tokens.
    - If None, no stop words will be used. max_df can be set to a value in the range [0.7, 1.0) to automatically detect and filter stop words based on intra corpus document frequency of terms.

In [ ]:
# remove English stop words

vect = CountVectorizer(stop_words='english')
tokenize_test(vect)

In [ ]:
# set of stop words

print vect.get_stop_words()

---

## Step 7: Other CountVectorizer Options to Raise Predictive Accuracy

- **max_features:** int or None, default=None
- If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.

In [ ]:
# remove English stop words and only keep 100 features

vect = CountVectorizer(stop_words='english', max_features=100)
tokenize_test(vect)

In [ ]:
# all 100 features

print vect.get_feature_names()

In [ ]:
# include 1-grams and 2-grams, and limit the number of features

vect = CountVectorizer(ngram_range=(1, 2), max_features=100000)
tokenize_test(vect)

In [ ]:
# include 1-grams and 2-grams

vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test(vect)

In [ ]:
# include 1-grams and 2-grams, and limit the number of features

vect = CountVectorizer(ngram_range=(1, 2), max_features=10000)
tokenize_test(vect)

- **min_df:** float in range [0.0, 1.0] or int, default=1
    - When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts.

In [ ]:
# include 1-grams and 2-grams, and only include terms that appear at least 2 times

vect = CountVectorizer(ngram_range=(1, 2),  max_features=30000, min_df=2)
tokenize_test(vect)

---

## Step 8: Using Term Frequency-Inverse Document Frequency (TF-IDF)

This is also a useful technique to distinguish material that does meet criteria versus text passages that contain omissions. Feel free to modify the code below to read in the raw_text column of the CSV you read in if you already understand what the sample scenario below illustrates. 

- **What:** Computes "relative frequency" that a word appears in a document compared to its frequency across all documents
- **Why:** More useful than "term frequency" for identifying "important" words in each document (high frequency in that document, low frequency in other documents)
- **Notes:** Used for search engine scoring, text summarization, document clustering

In [ ]:
# example instance, each string below is a hypothetical document

simple_train = ['call you tonight', 'Call me a cab', 'please call me... PLEASE!']

In [ ]:
# term frequency

vect = CountVectorizer()
tf = pd.DataFrame(vect.fit_transform(simple_train).toarray(), columns=vect.get_feature_names())
tf

In [ ]:
# document frequency

vect = CountVectorizer(binary=True)
df = vect.fit_transform(simple_train).toarray().sum(axis=0)
pd.DataFrame(df.reshape(1, 6), columns=vect.get_feature_names())

In [ ]:
# Term Frequency-Inverse Document Frequency (simple version)

tf/df

In [ ]:
# TfidfVectorizer

vect = TfidfVectorizer()
pd.DataFrame(vect.fit_transform(simple_train).toarray(), columns=vect.get_feature_names())

**More details:** [TF-IDF is about what matters](http://planspace.org/20150524-tfidf_is_about_what_matters/)

---
## Concluding Tips and Tricks

- It will always be easier to determine if an article *omits* something than screen how "true" a statement is
- This is all only as good as the criteria you set out to check articles and studies for
- That criteria is also a large influence on how good your training sample is
- Your workflow should check each of the criteria independently